In [1]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from Dataset.Economy_Dataset import Economy_Dataset
from Dataset.RNN_Transformer_Dataset import RNN_Transformer_Dataset

from Model.SelfAttention import Self_attention, Attention_LSTM
from Model.LSTM import LSTM

import matplotlib.pyplot as plt

In [3]:
# 데이터 다운 & 전처리

transaction_df = pd.read_excel('../데이터/transaction/transaction_final.xlsx',index_col=0)
economy_df = pd.read_excel('../데이터/economy/economy_all.xlsx')['국고채금리']

In [4]:
print(len(transaction_df))
print(len(economy_df))

300
204


In [5]:
# len_df = len(transaction_df)
# train_len = int(len_df *0.7)
# val_len = int(len_df * 0.3)

batch_size = 4
train_dataset = RNN_Transformer_Dataset(transaction_df, economy_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True)

# val_dataset = RNN_Transformer_Dataset(df[train_len:train_len+val_len])
# val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [6]:
for x,y,z,w in train_loader:
  print("X 크기 : {}".format(x.shape))
  print("Y 크기 : {}".format(y.shape))
  print("Z 크기 : {}".format(z.shape))
  print("W 크기 : {}".format(w.shape))


X 크기 : torch.Size([4, 158, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 158, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 158, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 158, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 158, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 158, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 158, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 158, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 158, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 158, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 158, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 158, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 158, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 158, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 158, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 158, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 158, 5])
Y 크기 : to

In [7]:
max_size = 158      #################
window_size = 5
hidden_size =256

In [9]:
# gpu 연결
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

# checkpoint 받기
eco_lstm = LSTM(window_size,256).to(device)
eco_lstm.load_state_dict(torch.load("../데이터/checkpoint/best_rnn_economy_model.pth"))

tran_lstm = LSTM(window_size,256).to(device)
tran_lstm.load_state_dict(torch.load("../데이터/checkpoint/best_rnn_transaction_model.pth"))

model = Self_attention(max_size, hidden_size, 1).to(device)

dummy = torch.rand((batch_size,max_size,hidden_size)).to(device)
model(dummy)

cpu is available


tensor([[[-0.4506],
         [-0.4402],
         [-0.4402],
         [-0.4379],
         [-0.4436],
         [-0.4407],
         [-0.4403],
         [-0.4445],
         [-0.4412],
         [-0.4445],
         [-0.4420],
         [-0.4443],
         [-0.4452],
         [-0.4440],
         [-0.4430],
         [-0.4498],
         [-0.4437],
         [-0.4435],
         [-0.4482],
         [-0.4468],
         [-0.4417],
         [-0.4425],
         [-0.4434],
         [-0.4438],
         [-0.4457],
         [-0.4472],
         [-0.4461],
         [-0.4437],
         [-0.4412],
         [-0.4420],
         [-0.4438],
         [-0.4439],
         [-0.4429],
         [-0.4454],
         [-0.4364],
         [-0.4467],
         [-0.4508],
         [-0.4464],
         [-0.4397],
         [-0.4451],
         [-0.4401],
         [-0.4454],
         [-0.4364],
         [-0.4403],
         [-0.4443],
         [-0.4466],
         [-0.4412],
         [-0.4503],
         [-0.4494],
         [-0.4382],


In [10]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

### 학습하기

In [20]:
train_n = len(train_dataset)
# val_n = len(val_dataset)

# val_loss_list = []

best_train_loss = float('inf')
stop_counter = 0
stop_criteria = 3
max_len = max_size

for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for tran_inputs, eco_inputs, tran_labels, eco_label in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함
        tran_inputs = tran_inputs.to(device) 
        eco_inputs = eco_inputs.to(device)
        tran_labels = tran_labels.to(device)  # batch_size*max_size*1
        eco_label = eco_label.to(device) 
        
        # transaction hidden_state + economy hidden_state
        # batch 단위로 한번에 더함
        # 콜금리 사용
        eco_tensor = eco_lstm(eco_inputs)[1][0].squeeze().unsqueeze(1).to(device)   # batch * hidden_state
        eco_tensor = eco_tensor.repeat(1,max_len,1)
        tran_tensor = torch.zeros((batch_size,max_len,hidden_size)).to(device)
        
        for i in range(max_len):
            tran_tensor[:,i,:] = tran_lstm(tran_inputs[:,i,:].unsqueeze(1))[1][0]
            
        emb_vector = tran_tensor + eco_tensor
        
        # 부동산 데이터가 0인 행은 embedding_vector도 0으로 전환  
        iszero_vector = torch.where(tran_labels.squeeze() == 0, tran_labels.squeeze(), 1.0)
           
        # 차원에 맞게 변경 후 계산 
        emb_vector = emb_vector.mul(iszero_vector.unsqueeze(2).repeat(1,1,hidden_size))
        
        #####################################
        # transformer 적용
        # loss 값은 부동산 가격이 0이 아닌 것에만 적용해서 학습
        out = model(emb_vector) # batch_size*max_size*1
        
        # out 값중 필요 없는 값은 0으로 변경
        out = out.mul(iszero_vector.unsqueeze(2))   
        
        labels = tran_labels.mul(iszero_vector.unsqueeze(2))
        
        # x값이 0이면 y값도 0인 상태!!!에서 아래 진행
        loss_all = 0
        for i in range(out.shape[1]):
            loss = criterion(out[:,i,:], labels[:,i,:])
            loss_all += loss
        loss_all = loss_all/iszero_vector.sum()   # 평균치만큼 업데이트!!!!!!!!
        
        mean_loss = loss_all/out.shape[1]
        optimizer.zero_grad() #
        mean_loss.backward() # loss가 최소가 되게하는
        optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
        running_loss += loss_all.item() # 한 배치의 loss 더해주고,

    if running_loss < best_train_loss:
        best_train_loss = running_loss
        torch.save(model.state_dict(), '../데이터/checkpoint/best_rnn_transformer_model.pth')

    print(running_loss/train_n)

    # with torch.no_grad():
    #         print("Calculating validation results...")
    #         model.eval()
    #         val_loss = 0
    #         for inputs, labels in val_loader:
    #             price, economy = inputs
    #             price = price.to(device) # batch_size*max_size*256  (max_size = 최대 단지 개수)
    #             economy = economy.to(device)
    #             labels = labels.to(device) # batch_size*max_size*1
                
    #             # labels 가 0이 아니면 lstm 넣기
    #             embed = torch.zeros((batch_size,max_size,hidden_size))
    #             for i in range(labels.shape[0]):
    #                 for j in range(labels.shape[1]):
    #                     if labels[i][j] != 0:
    #                         embed[i][j] = lstm_price(price[i][j]) 
                            
    #             outs = model(inputs)

    #             loss_all = 0
    #             for i in range(out.shape[1]):
    #                 loss = criterion(out[:,i,:], labels[:,i,:])
    #                 loss_all += loss

    #         avg_val_loss = val_loss/val_n/out.shape[1]
    #         val_loss_list.append(avg_val_loss)

    #         if avg_val_loss < best_val_loss:
    #             best_val_loss = avg_val_loss
    #             stop_counter = 0            
    #             torch.save(model.state_dict(), 'Checkpoint/economic_best_model.pth')
    #         else:
    #             stop_counter += 1

    #         print("val")
    #         print(
    #             f"loss: {avg_val_loss:4.8} || "
    #         )

    #         if stop_counter >= stop_criteria:
    #             print('Early stopping triggered.')
    #             break
    
    print('======================================================')

# plt.figure(figsize=(10, 5))
# plt.plot(val_loss_list, label='Validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

epoch : 0
train


KeyboardInterrupt: 